In [ ]:
from owslib.wms import WebMapService
import folium

In [ ]:
# Available years 2017-2021
year = '2021'
# Available sensors: S2, L8
sensor = 'S2'
# Available product names: SR, TIR, NDVI, NDWI
product = 'NDVI'
wms_endpoint = f'http://150.140.202.118:82/?map=/etc/mapserver/IR2M/{sensor}_{product}_{year}.map&service=WMS&version=1.3.0&request=GetCapabilities'

In [ ]:
wms = WebMapService(wms_endpoint, version='1.3.0')

In [ ]:
wms.identification.title

In [ ]:
[op.name for op in wms.operations]

In [ ]:
list(wms.contents)

In [ ]:
scene_id = 'd20210622_wms'

In [ ]:
layer = wms.contents[scene_id]

In [ ]:
layer.abstract

In [ ]:
bbox = layer.boundingBoxWGS84
print(bbox)

In [ ]:
%matplotlib inline
import os, sys
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

def get_map(wms, layer_name, bbox, filename, style=''):
    wms.getOperationByName('GetMap').formatOptions
    img = wms.getmap(layers=[layer_name],
                     styles=[style],
                     size=(600,300),
                     srs='EPSG:4326',
                     bbox=bbox,
                     format='image/png',
                     transparent=True)

    with open(filename,'wb') as tmpfile:
        tmpfile.write(img.read())

In [ ]:
m = folium.Map(location=[39.1, 20.9], zoom_start=8, tiles='OpenStreetMap')
folium.Rectangle(bounds=[[float(bbox[1]), float(bbox[0])], [float(bbox[3]), float(bbox[2])]]).add_to(m)
m

In [ ]:
get_map(wms,scene_id,bbox, 'rgb.png')
image1 = mpimg.imread('rgb.png')
fig = plt.figure(figsize=(12,7))
img1 = plt.imshow(image1, extent=bbox, aspect='auto')
plt.show()

In [ ]:
m = folium.Map(location=[39.1, 20.9], zoom_start=10, tiles='OpenStreetMap')

folium.raster_layers.WmsTileLayer(
    url=wms_endpoint,
    name=f'IR2MA {sensor} {product} Layer',
    fmt="image/png",
    layers=scene_id,
    attr=u"EOfarm © 2021",
    transparent=True,
    overlay=True,
    control=True,
).add_to(m)
folium.LayerControl().add_to(m)
m